In [ ]:
import requests
import bs4 as bs
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# Declared functions
## Retrieve links containing tags for magyar-peter in Magyar Nemzet Online

In [ ]:
# Function to retrieve the links to the articles on the given page
def retrieve_mp_links_mno(page=1):
    response = requests.get(f"https://magyarnemzet.hu/cimke/magyar-peter?page={page}")
    soup = bs.BeautifulSoup(response.text, "html.parser")
    # Find all <a> tags with the class "article-link-wrapper"
    links = soup.find_all("a", class_="article-link-wrapper")

    # Define the base URL
    base_url = "https://magyarnemzet.hu"

    # Extract the href attribute from each link and prepend the base URL
    urls = [base_url + link.get("href") for link in links]
    return urls


## Retrieve article content from the given URL

In [ ]:
# Function to retrieve article content from a given URL
def fetch_and_parse_article(article_url):
    try:
        soup = bs.BeautifulSoup(
            requests.get(article_url).content,
            "html.parser",
        )
        # Find the <app-article-text> block
        article_text_block = soup.find("app-article-text", class_="is-first")

        # Extract all text within <p> and <li> tags inside the article text block
        paragraphs = article_text_block.find_all(["p", "li"])
        article_text = "\n".join(
            [
                "- " + para.get_text(separator="\n", strip=True)
                if para.name == "li"
                else para.get_text(separator="\n", strip=True)
                for para in paragraphs
            ]
        )

        # Initialize variables
        title, lead, publish_date, source = None, None, None, None

        # Extract the title
        try:
            title = soup.find("h1", class_="title").get_text(strip=True)
        except Exception as e:
            print(f"Failed to fetch title from {article_url}: {e}")

        # Extract the lead
        try:
            lead = soup.find("h2", class_="lead").get_text(strip=True)
        except Exception as e:
            print(f"Failed to fetch lead from {article_url}: {e}")

        # Extract the publish date
        try:
            publish_date = soup.find("span", class_="publishdate").get_text(strip=True)
        except Exception as e:
            print(f"Failed to fetch publish date from {article_url}: {e}")

        # Extract the source
        try:
            source = (
                soup.find("span", class_="source")
                .get_text(strip=True)
                .replace("Forrás:", "")
                .strip()
            )
        except Exception as e:
            print(f"Failed to fetch source from {article_url}: {e}")

        return {
            "title": title,
            "lead": lead,
            "publish_date": publish_date,
            "source": source,
            "article_text": article_text,
            "url": article_url,
        }
    except Exception as e:
        print(f"Failed to fetch {article_url}:\n{e}")
        return None

# Actionable cells
## Retrieve article urls on the first 5 pages of MNO

In [ ]:
urls = []
for page in tqdm(range(1, 6)):
    urls.extend(retrieve_mp_links_mno(page))

## Retrieve article data and store it in a Dataframe

- Fetch and parse the articles returns a dictionary for each cell in the Series of urls
- If a dict is converted to a Series, the keys will be the column names
- The resulting DataFrame will have columns for each key in the dictionaries

In [ ]:
# the below function takes about 4-5 minutes to run.
df = (
    pd.Series(urls)
    .progress_apply(lambda x: fetch_and_parse_article(x))
    .apply(pd.Series)
)

  1%|▏         | 2/160 [00:01<01:49,  1.44it/s]

Failed to fetch title from https://magyarnemzet.hu/bayer-zsolt-blogja/2024/11/hazudol-petike: 'NoneType' object has no attribute 'get_text'
Failed to fetch lead from https://magyarnemzet.hu/bayer-zsolt-blogja/2024/11/hazudol-petike: 'NoneType' object has no attribute 'get_text'
Failed to fetch publish date from https://magyarnemzet.hu/bayer-zsolt-blogja/2024/11/hazudol-petike: 'NoneType' object has no attribute 'get_text'
Failed to fetch source from https://magyarnemzet.hu/bayer-zsolt-blogja/2024/11/hazudol-petike: 'NoneType' object has no attribute 'get_text'


  2%|▏         | 3/160 [00:02<02:01,  1.29it/s]

Failed to fetch source from https://magyarnemzet.hu/belfold/2024/11/penzugyi-kaosz-a-tisza-partnal-a-nyilvanosan-elerheto-adatok-alatamasztjak-radnai-mark-alelnok-szavait: 'NoneType' object has no attribute 'get_text'


  3%|▎         | 5/160 [00:04<02:48,  1.09s/it]

...

 99%|█████████▉| 158/160 [04:07<00:02,  1.17s/it]

Failed to fetch source from https://magyarnemzet.hu/sport/2024/11/mike-tyson-jake-paul-bundagyanu-hivatalos-valasz: 'NoneType' object has no attribute 'get_text'


100%|██████████| 160/160 [04:09<00:00,  1.08s/it]

Failed to fetch source from https://magyarnemzet.hu/kulfold/2024/11/olyan-tel-jon-amit-talan-soha-nem-felejtunk: 'NoneType' object has no attribute 'get_text'


100%|██████████| 160/160 [04:10<00:00,  1.57s/it]
